# Trabajo Tesis: Preprocesamiento faenas

### Librerias y bases

In [1]:
import os
import requests

import numpy as np
import pandas as pd

from tqdm import tqdm
from IPython.display import display, HTML

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.distance import geodesic
from geopy.distance import distance
from scipy.spatial import KDTree

In [3]:
#os.chdir('c:/Users/artur/OneDrive/Documents/TrabajoTesis') 
os.chdir('C:/Users/admin/OneDrive/Documents/TrabajoTesis')

In [4]:
df_faenas = pd.read_csv('Cluster/df_f_c_d.csv')

In [5]:
lt = pd.read_csv('Bases/lineas_transmision.csv')

In [6]:
lt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1052 entries, 0 to 1051
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   NOMBRE      1052 non-null   object 
 1   PROPIEDAD   1052 non-null   object 
 2   TENSION     1052 non-null   float64
 3   TIPO        1052 non-null   object 
 4   FECHA_CREA  1052 non-null   object 
 5   Latitude    1052 non-null   float64
 6   Longitude   1052 non-null   float64
dtypes: float64(3), object(4)
memory usage: 57.7+ KB


In [7]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from tqdm import tqdm
import requests

# -----------------------------
# Parameters
# -----------------------------
RADIUS_KM = 1   # radio para agrupar faenas cercanas
TOP_N = 2       # número de líneas más cercanas a considerar

# -----------------------------
# Helper functions
# -----------------------------
def haversine_distance(lat1, lon1, lat2, lon2):
    """Distancia en km entre dos coordenadas."""
    return geodesic((lat1, lon1), (lat2, lon2)).km

def osrm_travel_time(lat1, lon1, lat2, lon2):
    """Tiempo de viaje en minutos usando OSRM."""
    try:
        url = f"http://router.project-osrm.org/route/v1/driving/{lon1},{lat1};{lon2},{lat2}?overview=false"
        r = requests.get(url, timeout=10)
        data = r.json()
        if "routes" in data and len(data["routes"]) > 0:
            return data["routes"][0]["duration"] / 60  # segundos → minutos
    except:
        pass
    return None

# -----------------------------
# 1️⃣ Inicializar columnas
# -----------------------------
string_cols = ['lt_nombre', 'lt_propiedad', 'lt_tipo', 'lt_fecha_crea']
numeric_cols = ['lt_tension', 'lt_dist_km', 'lt_tiempo_min']

for col in string_cols:
    df_faenas[col] = pd.Series(dtype="object")
for col in numeric_cols:
    df_faenas[col] = np.nan

# -----------------------------
# 2️⃣ Asignar línea de transmisión más cercana
# -----------------------------
unassigned_mask = df_faenas['lt_tiempo_min'].isna()
pbar = tqdm(total=unassigned_mask.sum(), desc="Asignando línea de transmisión")

for idx, row in df_faenas[unassigned_mask].iterrows():
    lat0, lon0 = row['Latitud'], row['Longitud']

    # Calcular distancias directas a todas las líneas
    lt_copy = lt.copy()
    lt_copy['distance_km'] = lt_copy.apply(
        lambda x: haversine_distance(lat0, lon0, x['Latitude'], x['Longitude']), axis=1
    )
    top_lt = lt_copy.nsmallest(TOP_N, 'distance_km')

    # Calcular tiempo de viaje con OSRM
    top_lt['travel_time'] = top_lt.apply(
        lambda x: osrm_travel_time(lat0, lon0, x['Latitude'], x['Longitude']), axis=1
    )
    top_lt = top_lt.dropna(subset=['travel_time'])

    if top_lt.empty:
        df_faenas.at[idx, 'lt_tiempo_min'] = -1
        pbar.update(1)
        continue

    # Seleccionar la línea con menor tiempo
    best = top_lt.loc[top_lt['travel_time'].idxmin()]

    # Guardar información de la línea
    df_faenas.at[idx, 'lt_nombre'] = best['NOMBRE']
    df_faenas.at[idx, 'lt_propiedad'] = best['PROPIEDAD']
    df_faenas.at[idx, 'lt_tension'] = best['TENSION']
    df_faenas.at[idx, 'lt_tipo'] = best['TIPO']
    df_faenas.at[idx, 'lt_fecha_crea'] = best['FECHA_CREA']
    df_faenas.at[idx, 'lt_dist_km'] = best['distance_km']
    df_faenas.at[idx, 'lt_tiempo_min'] = best['travel_time']

    pbar.update(1)

pbar.close()
print("✅ Finished assigning nearest transmission line info to all mines.")


Asignando línea de transmisión: 100%|██████████| 21158/21158 [20:28:00<00:00,  3.48s/it]   

✅ Finished assigning nearest transmission line info to all mines.


In [8]:
df_faenas

,RutEmpresa,NombreEmpresa,CategoriaFaena,RegionInstalacion,TipoInstalacion,Cota,Huso,Estado,Latitud,Longitud,...,Desal_Tipo,Desal_Capacidad_Ls,Desal_Año,lt_nombre,lt_propiedad,lt_tipo,lt_fecha_crea,lt_tension,lt_dist_km,lt_tiempo_min
0,61703000-4,EMPRESA NACIONAL DE MINERIA (ENAMI),CATEGORIA B,ATACAMA,TALLERES Y MAESTRANZA,538,19,ACTIVA,-28.566725,-70.740064,...,Plantas deslinizadoras de agua de mar con capa...,70.0,1997,VALLENAR - EL EDEN 110KV,CGE TRANSMISIÓN S.A.,ZONAL,2019-04-26,110.0,3.860702,8.480000
1,76125921-0,CUPRUM RESOURCES CHILE SPA,CATEGORIA D,COQUIMBO,EXPLORACION DE SUPERFICIE,1200,19,INACTIVA,-29.432920,-70.683699,...,Iniciativas y proyectos en evaluación preliminar,970.0,ND,PUNTA COLORADA - TRES QUEBRADAS 220KV,COMPAÑÍA BARRICK CHILE GENERACIÓN SPA,S/I,2019-04-26,220.0,8.756235,65.498333
2,76362163-4,MINERA RUPERTO PATRICIO ORTIZ PALOMINOS,CATEGORIA D,LIBERTADOR GENERAL BERNARDO OHIGGINS,MINA SUBTERRANEA,134,19,ABANDONADA,-34.677189,-71.498091,...,Plantas deslinizadoras de agua de mar con capa...,30.0,2008,PANIAHUE - SANTA CRUZ 66KV,CGE TRANSMISIÓN S.A.,ZONAL,2023-04-10,66.0,14.601320,19.891667
3,6891906-1,GARY NELSON MERCADO MALBRAN,CATEGORIA D,ATACAMA,MINA SUBTERRANEA,531,19,ACTIVA,-26.411292,-70.414104,...,Plantas deslinizadoras de agua de mar con capa...,120.0,2014,EL SALADO - CHANARAL 23KV,TRANSELEC S.A.,ZONAL,2017-12-20,110.0,5.681348,5.305000
4,76448510-6,SLM CASUALIDAD PRIMERA DE EL MANZANO,CATEGORIA D,COQUIMBO,MINA SUBTERRANEA,917,19,IRREGULAR OPERATIVA,-30.204069,-71.082232,...,Iniciativas y proyectos en evaluación preliminar,970.0,ND,TAP SDGX01 - SDGX01 23KV,ENGIE ENERGÍA CHILE S.A.,ZONAL,2023-04-10,23.0,3.013159,11.110000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21153,78131832-9,BALFENI SOLUTIONS SPA,CATEGORIA D,COQUIMBO,CAMPAMENTO,1277,19,EN REVISIÓN DE PROYECTO,-31.216132,-71.267759,...,Plantas en construcción o prontas a entrar en ...,400.0,2023,OVALLE - ILLAPEL 110KV,CGE TRANSMISIÓN S.A.,ZONAL,2019-04-26,110.0,16.721292,54.318333
21154,78131832-9,BALFENI SOLUTIONS SPA,CATEGORIA D,COQUIMBO,MINA SUBTERRANEA,1199,19,EN REVISIÓN DE PROYECTO,-31.220795,-71.265226,...,Plantas en construcción o prontas a entrar en ...,400.0,2023,OVALLE - ILLAPEL 110KV,CGE TRANSMISIÓN S.A.,ZONAL,2019-04-26,110.0,16.938373,58.196667
21155,6188182-4,EDILIO GOMEZ OYARZUN,CATEGORIA D,COQUIMBO,MINA SUBTERRANEA,458,19,EN REVISIÓN DE PROYECTO,-31.548972,-71.343902,...,Plantas en construcción o prontas a entrar en ...,400.0,2023,LOS VILOS - LAS PALMAS 220KV,TRANSELEC S.A.,NACIONAL,2019-04-26,220.0,19.032049,64.020000
21156,78110627-5,C & C MINERIA SpA,CATEGORIA D,ANTOFAGASTA,POLVORIN,3250,19,EN REVISIÓN DE PROYECTO,-23.044930,-68.557016,...,Plantas con evaluación ambiental aprobada y co...,140.0,2024,LASANA - SOLAR JAMA 220KV,PLANTA SOLAR SAN PEDRO III SPA,DEDICADO,2019-04-26,220.0,56.838178,92.448333


In [9]:
df_faenas.to_csv('Cluster/df_f_c_d_lt.csv', index=False)